In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime



In [2]:
def get_data_text(result):
    list_items_text = [li.text for li in result.find_all('li')]
    if list_items_text and len(list_items_text) > 1:
        # remove duplicities
        return ', '.join(list(set(list_items_text)))
    return result.text


In [3]:
url = 'https://eventsmadeira.com/en/event-listing/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')
divs = soup.find_all('div', class_='gt-event-style-3')


In [65]:
fields = ["title", "date", "time", "location", "category"]

data = {field: [] for field in fields}

for div in divs:
    for field in fields:
        
        result = div.find('div', class_=f'gt-{field}')
        if result is not None: 
            data[field].append(get_data_text(result))
        else: 
            data[field].append('-')


In [66]:
fields = ["start_date", "end_date", "start_time", "end_time"]
# todo: do not introduce new fields, but modify the existing ones
data_dt = {field: [] for field in fields}

for date, time in zip(data['date'], data['time']): 
    splitted_date = date.split(' - ')
    if len(splitted_date) > 1:
        # todo: handle case where time is interval (e.g. 10:00 - 12:00)
        start = datetime.strptime(splitted_date[0] + time, '%d/%m/%Y%H:%M')
        end = datetime.strptime(splitted_date[1], '%d/%m/%Y')
        data_dt['start_date'].append(start.date())
        data_dt['end_date'].append(end.date())
        data_dt['start_time'].append(start.time())
        data_dt['end_time'].append(end.time())
    else:
        if splitted_date[0] == '-': 
            data_dt['start_date'].append('-')
            data_dt['end_date'].append('-')
            data_dt['start_time'].append('-')
            data_dt['end_time'].append('-')
        else: 
            start = datetime.strptime(splitted_date[0], '%d/%m/%Y')
            data_dt['start_date'].append(start.date())
            data_dt['end_date'].append('-')
            data_dt['start_time'].append(start.time())
            data_dt['end_time'].append('-')


        


In [73]:

df = pd.DataFrame(data)
df_dt = pd.DataFrame(data_dt)
combined_df = pd.concat([df, df_dt], axis=1).drop(["date", "time"], axis=1)

combined_df


,title,location,category,start_date,end_date,start_time,end_time
0,Nativity scenes around the island,"Câmara de Lobos, São Vicente, Santa Cruz, Pont...",Christmas,-,-,-,-
1,Rampa dos Barreiros,Funchal,"Antiques, Sports",-,-,-,-
2,Arrival of Santa Claus’s Allegoric Parade,Câmara de Lobos,Christmas,-,-,-,-
3,“Waterline” exhibition,Funchal,Culture,2023-06-20,2028-06-20,10:00:00,00:00:00
4,Conventual sweets Route,Funchal,Gastronomy,2023-11-03,2023-11-24,10:00:00,00:00:00
...,...,...,...,...,...,...,...
71,Madeira Wine Lounge,Funchal,Gastronomy,2024-08-29,2024-09-15,18:00:00,00:00:00
72,Concerts in the Vineyards,"São Vicente, Calheta, Ribeira Brava","Gastronomy, Music",2024-09-07,2024-09-15,17:00:00,00:00:00
73,Columbus Festival,Porto Santo,"Antiques, Official Poster",2024-09-19,2024-09-22,15:00:00,00:00:00
74,World Tourism Day,"Ponta do Sol, Funchal, Machico, Porto Moniz, P...",Other,2024-09-27,-,00:00:00,-
